# Zmiany semantyczne słów

<img src="https://live.staticflickr.com/65535/54942563983_f3baea0eee_c.jpg" alt="Embedded Photo" width="776">

## Wstęp

Znaczenia słów w języku naturalnym zmieniają się wraz z czasem — jedne ewoluują stopniowo, inne przechodzą gwałtowne przesunięcia semantyczne. Dzięki dawnym korpusom tekstowym możliwe jest trenowanie modeli wektorowych, które uchwytują te historyczne różnice.

W tym zadaniu będziesz analizować **pretrenowane embeddingi (word2vec)** z dwóch różnych epok:
- **1900** — model wytrenowany wyłącznie na danych z początku XX wieku  
- **1990** — model wytrenowany na danych bliskich współczesności  

Te dwa modele zostały wytrenowane **całkowicie niezależnie**. Twoim zadaniem będzie wykorzystać te reprezentacje, aby zbudować klasyfikator określający, czy **znaczenie danego słowa uległo istotnej zmianie między rokiem 1900 a 1990**.

## Zadanie

Zbudujesz **klasyfikator binarny**, który dla zadanego słowa zwróci etykietę:

- **0 — słowo stabilne semantycznie**
- **1 — słowo, którego znaczenie się zmieniło**

## Dane

- `train.csv` — zbiór treningowy (słowo + etykieta)
- `valid.csv` — zbiór walidacyjny do lokalnego testowania
- `1900-vocab.pkl` oraz `1900-w.npy` — słownik i macierz embeddingów z 1900 roku  
- `1990-vocab.pkl` oraz `1990-w.npy` — analogiczny zestaw dla roku 1990  

## Kryterium oceny

Do oceny Twojego rozwiązania wykorzystujemy metrykę **Balanced Accuracy**, czyli średnią dokładności klasyfikacji dla klasy pozytywnej i dla klasy negatywnej. Innymi słowy:

$$ \text{Balanced Accuracy} = \frac{1}{2}(\text{TPR} + \text{TNR}) $$

czyli średnią z czułości (TPR) i specyficzności (TNR).  
Jest to metryka odporna na niezbalansowanie zbiorów danych.

Zwracasz **twarde etykiety (0/1)**.

W notebooku znajduje się funkcja `evaluate_algorithm`, dzięki której przetestujesz swój model na `valid.csv`.

Za to zadanie możesz zdobyć pomiędzy 0 a 100 punktów. Wynik będzie skalowany liniowo w zależności od wartości Balanced Accuracy:

- **Balanced Accuracy ≤ 0.7**: 0 punktów.
- **Balanced Accuracy ≥ 0.87**: 100 punktów.
- **Wartości pomiędzy 0.7 a 0.87**: skalowane liniowo.

Wzór na wynik:  
$$
\text{Punkty} = 
\begin{cases} 
0 & \text{dla } \text{Balanced Accuracy} \leq 0.7 \\
100 \times \frac{\text{Balanced Accuracy} - 0.7}{0.87 - 0.7} & \text{dla } 0.7 < \text{Balanced Accuracy} < 0.87 \\
100 & \text{dla } \text{Balanced Accuracy} \geq 0.87
\end{cases}
$$

## Ograniczenia

Twój notebook będzie uruchamiany na Platformie Konkursowej:

- **bez dostępu do internetu**
- bez dostępu do GPU - **CPU only**
- Limit czasu wykonania notebooka i ewaluacji na zbiorze testowym: **5 minut**
- Lista dopuszczalnych bibliotek: `numpy`, `pandas`, `scikit-learn`, `matplotlib`, `tqdm`

## Pliki zgłoszeniowe

- Ten notebook uzupełniony o Twoje rozwiązanie:

```python
class SemanticChangeModel:
    def fit(self, train_df):
        ...
    def predict_change(self, words: List[str]) -> List[int in {0,1}]:
        ...
```

## Ewaluacja

Pamiętaj, że podczas sprawdzania flaga `FINAL_EVALUATION_MODE` zostanie ustawiona na `True`.

Za to zadanie możesz zdobyć pomiędzy 0 a 100 punktów. Liczba punktów, którą zdobędziesz, będzie wyliczona na (tajnym) zbiorze testowym na Platformie Konkursowej na podstawie wyżej wspomnianego wzoru, zaokrąglona do liczby całkowitej. Jeśli Twoje rozwiązanie nie będzie spełniało powyższych kryteriów lub nie będzie wykonywać się prawidłowo, otrzymasz za zadanie 0 punktów.

## Kod Startowy
W tej sekcji inicjalizujemy środowisko poprzez zaimportowanie potrzebnych bibliotek i funkcji. Przygotowany kod ułatwi Tobie efektywne operowanie na danych i budowanie właściwego rozwiązania.

In [1]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################

FINAL_EVALUATION_MODE = False  # Podczas sprawdzania ustawimy tę flagę na True.

import os, json, pickle, shutil
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

from sklearn.metrics import balanced_accuracy_score

np.random.seed(42)
random.seed(42)

DATA_DIR = Path('data')
EMB_DIR  = DATA_DIR / 'embeddings'
EMB_DIR.mkdir(parents=True, exist_ok=True)


## Pobieranie danych (tylko lokalnie)

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################

# Uwaga: Na platformie oceniającej Internet jest wyłączony.
# Ten blok uruchamia się tylko lokalnie (gdy FINAL_EVALUATION_MODE == False).

GDRIVE_FILES = [
    ('1rUGgDZcpwRZ5sRHGxxEh2f7ZJ0DRVDPL', EMB_DIR / '1900-vocab.pkl'),
    ('1cYXPhghcawbMZ6vU2XyJUq7NOpBIKj5E', EMB_DIR / '1900-w.npy'),
    ('1ApLkBn2ylvLMKNlNtvkMVde6RxnLJolI', EMB_DIR / '1990-vocab.pkl'),
    ('1B3NLInA4Ty3lUaHNQgxtDTKJNtG0t0T1', EMB_DIR / '1990-w.npy'),
    ('1hrOfZOq3BV1K0tWe6HSZG-OiZkGlCiYT', DATA_DIR / 'train.csv'),
    ('1vndyCuDCBP6zLvNkF_YsKHgTQgulTjt_', DATA_DIR / 'valid.csv'),
]

def download_data():
    try:
        import gdown
    except Exception as e:
        raise RuntimeError('Zainstaluj gdown lokalnie: `pip install gdown`') from e

    DATA_DIR.mkdir(parents=True, exist_ok=True)
    EMB_DIR.mkdir(parents=True, exist_ok=True)

    for fid, out_path in GDRIVE_FILES:
        if out_path.exists():
            print(f'Pominięto pobieranie — plik już istnieje: {out_path.name}')
            continue
        url = f'https://drive.google.com/uc?id={fid}'
        out_path.parent.mkdir(parents=True, exist_ok=True)
        print(f'Pobieranie -> {out_path.name}')
        gdown.download(url, str(out_path), quiet=False)

if not FINAL_EVALUATION_MODE:
    download_data()
    print('Pobieranie zakończone.')
else:
    print('FINAL_EVALUATION_MODE=True — pomijam pobieranie (na platformie dane są dostarczone).')

## Ładowanie embeddingów i zbiorów danych

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################

def load_histwords_decade(decade: int, emb_dir: Path):
    vocab_path = emb_dir / f'{decade}-vocab.pkl'
    w_path     = emb_dir / f'{decade}-w.npy'
    with open(vocab_path, 'rb') as f:
        vocab = pickle.load(f)
    W = np.load(w_path)
    # Normalizacja L2
    W = W / (np.linalg.norm(W, axis=1, keepdims=True) + 1e-12)
    w2i = {w:i for i,w in enumerate(vocab)}
    return vocab, W, w2i

# Wczytanie embeddingów
vocab_1900, W1900, w2i_1900 = load_histwords_decade(1900, EMB_DIR)
vocab_1990, W1990, w2i_1990 = load_histwords_decade(1990, EMB_DIR)

if not FINAL_EVALUATION_MODE:
    print(f'1900: V={len(vocab_1900):,}, dim={W1900.shape[1]}')
    print(f'1990: V={len(vocab_1990):,}, dim={W1990.shape[1]}')

# Wczytanie zbiorów train/valid
train_path = DATA_DIR / 'train.csv'
valid_path = DATA_DIR / 'valid.csv'
assert train_path.exists() and valid_path.exists(), 'Brak train.csv / valid.csv w folderze data/'

train_df = pd.read_csv(train_path)
valid_df = pd.read_csv(valid_path)

# Oczekiwane kolumny: word, label
for c in ['word', 'label']:
    assert c in train_df.columns and c in valid_df.columns, 'Oczekiwane kolumny: word, label'

train_df['word'] = train_df['word'].astype(str).str.lower().str.strip()
valid_df['word'] = valid_df['word'].astype(str).str.lower().str.strip()
train_df['label'] = train_df['label'].astype(int)
valid_df['label'] = valid_df['label'].astype(int)


if not FINAL_EVALUATION_MODE:
    print(train_df.head(5))
    print()
    print(valid_df.head(5))
    print(f'train: {len(train_df)}, valid: {len(valid_df)}')

1900: V=100,000, dim=300
1990: V=100,000, dim=300
        word  label
0     lichen      0
1    imaging      1
2      devil      0
3    prayers      0
4  frankfort      1

        word  label
0  coastline      0
1       yoke      0
2     report      1
3   language      0
4       barn      0
train: 2495, valid: 832


In [ ]:
def print_neighbors(word, V, vocab, w2i, label):
    vec = V[w2i[word]]
    sims = V @ vec
    sims[w2i[word]] = -np.inf
    top = np.argsort(-sims)[:10]
    print(f"\nTop 10 sąsiadów w {label}:")
    for i in top:
        print(f"  {vocab[i]}  ({sims[i]:.4f})")


if not FINAL_EVALUATION_MODE:
    print_neighbors("intelligence", W1900, vocab_1900, w2i_1900, "1900")
    print_neighbors("intelligence", W1990, vocab_1990, w2i_1990, "1990")


Top 10 sąsiadów w 1900:
  intellect  (0.4798)
  sagacity  (0.4282)
  discernment  (0.3707)
  understanding  (0.3688)
  tact  (0.3659)
  skill  (0.3643)
  humanity  (0.3641)
  honesty  (0.3598)
  ability  (0.3517)
  sensibility  (0.3512)

Top 10 sąsiadów w 1990:
  iq  (0.3773)
  wechsler  (0.3542)
  cia  (0.3362)
  hinsley  (0.3301)
  artificial  (0.3264)
  afric  (0.3202)
  binet  (0.3191)
  aptitude  (0.3152)
  abwehr  (0.3108)
  abilities  (0.3070)


## Twoje rozwiązanie

In [5]:
#########################  ZMODYFIKUJ TYLKO TĘ KOMÓRKĘ  #########################
# Zaimplementuj swój model jako klasę z metodami:
#   - __init__       : zapisz osadzenia (embeddings) + podstawowe hiperparametry
#   - fit(train_df)  : trenuj na oznaczonych danych
#   - predict_change(words) : zwróć etykiety dla podanej listy słów
#
# Kod ewaluacyjny otrzyma instancję klasy i będzie zakładał jedynie, że posiada
# metodę .predict_change(words).

class SemanticChangeModel:
    def __init__(self, W1900, W1990, w2i_1900, w2i_1990):
        """
        Przechowuj tutaj wszystkie kosztowne / globalne obiekty. Możesz zbudować
        dodatkowe struktury w metodzie fit().

        Parametry
        ----------
        W1900, W1990 : np.ndarray [V, D]
            Znormalizowane osadzenia (embeddings) dla lat 1900 i 1990.
        w2i_1900, w2i_1990 : dict
            Mapowanie słowo -> indeks wiersza w osadzeniach.
        """
        self.W1900 = W1900
        self.W1990 = W1990
        self.w2i_1900 = w2i_1900
        self.w2i_1990 = w2i_1990

        # Możesz dodać więcej parametrów i metod według potrzeb

    def fit(self, train_df):
        """
        Zbuduj swój model, korzystając z oznaczonych danych treningowych.

        Parametry
        ----------
        train_df : pd.DataFrame
            Musi zawierać przynajmniej kolumny ['word', 'label'].
        """
        # TODO: zastąp ten placeholder swoją rzeczywistą logiką dopasowywania.
        pass
    
    def predict_change(self, words):
        """
        Przewiduje, czy słowa znacząco zmieniły swoje znaczenie.

        Parametry
        ----------
        words : list of str
            Lista słów do sklasyfikowania.

        Zwraca
        -------
        list of int
            Lista 0 lub 1 (1 = 'zmienione') dla każdego słowa.
        """
        # TODO: zastąp ten placeholder swoją rzeczywistą logiką predykcji.
        return [random.choice([0,1]) for _ in words]


MODEL = SemanticChangeModel(W1900, W1990, w2i_1900, w2i_1990)
MODEL.fit(train_df)


## Ewaluacja

Uruchomienie poniższej komórki pozwoli sprawdzić, ile punktów zdobyłoby Twoje rozwiązanie na danych walidacyjnych.

Upewnij się przed wysłaniem, że cały notebook wykonuje się od początku do końca bez błędów i bez ingerencji użytkownika po wykonaniu polecenia `Run All`.

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################
def compute_score(bal_acc: float) -> float:
    """
    Oblicza wynik punktowy na podstawie wartości zbalansowanej dokładności.

    :param bal_acc: Wartość float w zakresie [0.0, 1.0]
    :return: Wynik punktowy zgodny z określoną funkcją
    """
    if bal_acc <= 0.7:
        return 0
    elif 0.7 < bal_acc < 0.87:
        return int(round(100 * (bal_acc - 0.7) / (0.87 - 0.7)))
    else:
        return 100


def evaluate_algorithm(dataset_df, model, verbose=False):
    """
    Ewaluacja modelu wykrywania zmiany znaczenia słów na podanym zbiorze danych.

    Parametry
    ----------
    dataset_df : pd.DataFrame
        Oznaczony zbiór danych z kolumnami:
          - 'word'  : słowo (string)
          - 'label' : etykieta 0 = stabilne, 1 = zmienione

    model : obiekt
        Obiekt posiadający metodę:
            predict_change(words: list[str]) -> list[int] {0,1}

    verbose : bool
        Jeśli True, wypisuje dodatkowe informacje.

    Zwraca
    -------
    points : float
        Wynik punktowy oparty na zbalansowanej dokładności.
    """

    # Wyodrębnij słowa i etykiety z datasetu
    words = dataset_df["word"].astype(str).tolist()
    ys = dataset_df["label"].astype(int).tolist()

    # Pobierz predykcje dla całej listy słów
    preds = model.predict_change(words)

    # Konwersja predykcji i etykiet na tablice numpy
    preds = np.array(preds, dtype=np.int32)
    ys = np.array(ys, dtype=np.int32)

    # Zbalansowana dokładność
    bal_acc = balanced_accuracy_score(ys, preds)

    # Konwersja dokładności na punkty konkursowe
    points = compute_score(bal_acc)

    if verbose:
        print(f"\nLiczba próbek: {len(dataset_df)}")
        print(f"Balanced accuracy: {bal_acc:.4f}")
        print(f"Wynik punktowy: {points}")

    return points


if not FINAL_EVALUATION_MODE:
    _ = evaluate_algorithm(valid_df, MODEL, verbose=True)


Liczba próbek: 832
Balanced accuracy: 0.5157
Wynik punktowy: 0
